# Extraccion de data

In [66]:
#importando librerias
import requests
from bs4 import BeautifulSoup
import numpy as np

In [77]:
#url del producto
url= "https://hiraoka.com.pe/celulares/celulares/celular-libre-samsung-galaxy-a15-6-5-256gb-8gb-ram-blue-black"

##El header es información que mandan los navegadores, debemos incluirlo para simular que somos un usuario normal
header = {"User-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36"}


In [78]:
#pedicion para visitar la pagina
r= requests.get(url, headers=header, timeout=10)
r.status_code

200

In [79]:
#soup
soup= BeautifulSoup(r.text,'html.parser')

In [80]:
#se almacenara en una lista
producto= list()


In [81]:
#buscador de nombre del producto
nom_producto=soup.find('h1', class_= "page-title").text
producto.append(nom_producto)


In [82]:
#buscador precio del producto
precio_producto= soup.find('span', attrs={'class':'price'}).text
producto.append(precio_producto)

In [83]:
producto

['\nCelular Libre Samsung Galaxy A15 6.5" 256GB 8GB RAM Blue Black ',
 'S/ \xa0699.00']

# Limpieza de data 

In [99]:
def clean_data (data_product):
    #eliminando caracteres especiales
    for i in range(len(data_product)):
        if  data_product[i]  is str: 
            data_product[i] = data_product[i].replace("\n","",).replace("S/","").strip()
        
    #cambiando el precio del producto a float
    if type(data_product[1]) is str :
        data_product[1]= np.float(data_product[1])
    
    return data_product

In [100]:
producto = clean_data(producto)
producto

['Celular Libre Samsung Galaxy A15 6.5" 256GB 8GB RAM Blue Black', 699.0]

# Enviar por correo

In [138]:
# se crea una función para enviar el correo 
# parametros de entrada: 
# - asunto del correo (subject)
# - contenido del correo (body)
# - los datos de la cuenta de correo electrónico Gmail (user)
# - la contraseña de acceso del correo (pwd) 
# - la dirección de correo electrónico a la que queremos notificar (recipient)

def send_email(subject, body, user='email09@hotmail.com', pwd='password', recipient='cuya210@gmail.com'):
    
    import smtplib
    from email.mime.text import MIMEText

    #contenido del correo
    msg = MIMEText(body)
    
    #el asunto del correo
    msg['Subject'] = subject
    
    #remitente del correo
    msg['From'] = user
    
    #correo destino
    msg['To'] = recipient


    try:
        #se utiliza with para que se cierre atmaticamente la conexión con el servidor SMTP
        with smtplib.SMTP("smtp-mail.outlook.com", port=587) as server:
            
            #se envía el comando EHLO al servidor SMTP para identificar al cliente y solicitar capacidades adicionales 
            #inicia la comunicación con el servidor.)
            server.ehlo()
            
            #se inicia el cifrado TLS para asegurar la conexión entre tu aplicación y el servidor SMTP
            #asegura la conexión cifrando la comunicación.
            server.starttls()
            
            #autentica al usuario con su nombre de usuario y contraseña
            server.login(user, pwd)
            
            #se envia el correo
            server.sendmail(user, recipient, msg.as_string())
            
        print('Se envió el correo correctamente')
        
    except smtplib.SMTPAuthenticationError:
        print('Error de autenticación')
        
    except smtplib.SMTPRecipientsRefused:
        print('El destinatario rechazó el mensaje')
 
    except smtplib.SMTPSenderRefused:
        print('El remitente fue rechazado')
        
    except smtplib.SMTPException as e:
        print(f'Error SMTP: {e}')
        
    except Exception as e:
        print(f'Fallo el envío de correo: {e}')
        

# Creación de la alerta

In [139]:
precio_alerta= 700.00

if producto[1] < precio_alerta:
    #se prepara el encabezado y cuerpo del correo
    subject= f"Alerta Baja de Precio del Producto {producto[0]}"
    body= f"\n El producto {producto[0]} ha bajado de precio \n Antes costaba S/.{precio_alerta} ahora S/. {producto[1]} \n Visita la pagina {url}"
    send_email(subject, body) 

Se envió el correo correctamente
